In [21]:
# with ganache installed "npm install ganache --global"
# run "ganache ethereum -d" which will start a local ethereum node in deterministic mode so that the same accounts are generated each time

# deploying to mainnet or testnet is pretty much exactly the same as this. we just need to use an external blockchain node instead of ganache
# Infura is a good service for this. https://infura.io/ and it's free to start with
# what you would do is when you are creating the web3 object, you would use the infura endpoint instead of the ganache endpoint
# and use a real wallet address with a real private key instead of the ganache wallet address and private key

In [1]:
# now we can do much the same as we have in the deploy.ipynb notebook

In [2]:
with open("../contracts/SimpleStorage.sol", "r") as file:
    simple_storage_file = file.read()

print(simple_storage_file)

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.12;

contract SimpleStorage {

    // this will get initialized to 0
    uint256 favoriteNumber;
    bool favoriteBool;

    struct Person {
        uint256 favoriteNumber;
        string name;
    }

    // this array is dynamic specifying
    Person[] public people;
    // mappings in sol have default values, for uint256 it's 0
    mapping(string => uint256) public nameToFavoriteNumber;

    function store(uint256 _favoriteNumber) public {
        favoriteNumber = _favoriteNumber;
    }

    function retrieve() public view returns (uint256) {
        return favoriteNumber;
    }

    function addPerson(string memory _name, uint256 _favoriteNumber) public {
        people.push(Person(_favoriteNumber, _name));
        nameToFavoriteNumber[_name] = _favoriteNumber;
    }

}

// what if we want to have many instances of this contract?



In [4]:
# now we have to compile our smart contract - will use python solc-x package
from solcx import compile_standard

In [5]:
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SimpleStorage.sol": {"content": simple_storage_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.17",
)


In [6]:
# get bytecode
bytecode = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["evm"]["bytecode"]["object"]
# get abi
abi = compiled_sol["contracts"]["SimpleStorage.sol"]["SimpleStorage"]["abi"]

In [7]:
# now deploy

from web3 import Web3

# connect to local ganache
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
chain_id = 1337
my_address = "0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1"
private_key = "0x4f3edf983ac636a65a842ce7c78d9aa706d3b113bce9c46f30d7d21715b23b1d"

In [8]:
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)
SimpleStorage

web3._utils.datatypes.Contract

In [9]:
# build a transaction that deploys the contract
# get the latest transaction nonce
nonce = w3.eth.getTransactionCount(my_address)
nonce

0

In [10]:
# build a transaction - SimpleStorage does not have a constructor
transaction = SimpleStorage.constructor().buildTransaction({
    "gasPrice": w3.eth.gas_price,
    "chainId": chain_id,
    "from": my_address,
    "nonce": nonce,
})
transaction

{'value': 0,
 'gas': 562541,
 'gasPrice': 2000000000,
 'chainId': 1337,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'nonce': 0,
 'data': '0x608060405234801561001057600080fd5b5061093b806100206000396000f3fe608060405234801561001057600080fd5b50600436106100575760003560e01c80632e64cec11461005c5780636057361d1461007a5780636f760f41146100965780638bab8dd5146100b25780639e7a13ad146100e2575b600080fd5b610064610113565b60405161007191906102b2565b60405180910390f35b610094600480360381019061008f919061030d565b61011c565b005b6100b060048036038101906100ab9190610480565b610126565b005b6100cc60048036038101906100c791906104dc565b6101af565b6040516100d991906102b2565b60405180910390f35b6100fc60048036038101906100f7919061030d565b6101dd565b60405161010a9291906105a4565b60405180910390f35b60008054905090565b8060008190555050565b6002604051806040016040528083815260200184815250908060018154018082558091505060019003906000526020600020906002020160009091909190915060008201518160000155602082015181600101908161018591906107e0565b5050

In [11]:
# sign the transaction
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)
signed_txn

SignedTransaction(rawTransaction=HexBytes('0xf909af8084773594008308956d8080b9095b608060405234801561001057600080fd5b5061093b806100206000396000f3fe608060405234801561001057600080fd5b50600436106100575760003560e01c80632e64cec11461005c5780636057361d1461007a5780636f760f41146100965780638bab8dd5146100b25780639e7a13ad146100e2575b600080fd5b610064610113565b60405161007191906102b2565b60405180910390f35b610094600480360381019061008f919061030d565b61011c565b005b6100b060048036038101906100ab9190610480565b610126565b005b6100cc60048036038101906100c791906104dc565b6101af565b6040516100d991906102b2565b60405180910390f35b6100fc60048036038101906100f7919061030d565b6101dd565b60405161010a9291906105a4565b60405180910390f35b60008054905090565b8060008190555050565b6002604051806040016040528083815260200184815250908060018154018082558091505060019003906000526020600020906002020160009091909190915060008201518160000155602082015181600101908161018591906107e0565b5050508060038360405161019991906108ee565b90815260200160405180910390208190555

In [12]:
# send the transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)

In [13]:
# wait for the transaction to be mined
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x7b7097853f6303f2cf0a307050ab7eedb8b384351246ad850a7eaac4127bb8f0'),
 'transactionIndex': 0,
 'blockNumber': 1,
 'blockHash': HexBytes('0x52eac9959555aa2de94a91821c01408208fdafb9846ffd13ac35ac0951b532d6'),
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': None,
 'cumulativeGasUsed': 562541,
 'gasUsed': 562541,
 'contractAddress': '0xe78A0F7E598Cc8b0Bb87894B0F60dD2a88d6a8Ab',
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status':

In [14]:
# working with the contract
# remember we always need the address and abi

simple_storage = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)

In [15]:
simple_storage.all_functions()

[<Function addPerson(string,uint256)>,
 <Function nameToFavoriteNumber(string)>,
 <Function people(uint256)>,
 <Function retrieve()>,
 <Function store(uint256)>]

In [16]:
simple_storage.functions.retrieve().call()

0

In [17]:
# now store a new value

store_txn = simple_storage.functions.store(15).buildTransaction({
    "gasPrice": w3.eth.gas_price,
    "chainId": chain_id,
    "from": my_address,
    "nonce": nonce + 1,
})

store_txn

{'value': 0,
 'gas': 43724,
 'gasPrice': 2000000000,
 'chainId': 1337,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'nonce': 1,
 'to': '0xe78A0F7E598Cc8b0Bb87894B0F60dD2a88d6a8Ab',
 'data': '0x6057361d000000000000000000000000000000000000000000000000000000000000000f'}

In [18]:
# sign the transaction
signed_store_txn = w3.eth.account.sign_transaction(store_txn, private_key=private_key)
signed_store_txn

SignedTransaction(rawTransaction=HexBytes('0xf88901847735940082aacc94e78a0f7e598cc8b0bb87894b0f60dd2a88d6a8ab80a46057361d000000000000000000000000000000000000000000000000000000000000000f820a95a0c664c48348dfc8db7b5849fab84324a77e585a9b90932d2741838b4751f38019a0728b85fec8d9a9da9d5bf2ba7998f80472de29bd93dfd0f8c382cce3b0babcac'), hash=HexBytes('0x79a778ba5f89a102853db07a34f88572b42c8c1942a96f956c11d2ba5ee57e1e'), r=89735985007676112322637977936966552555421687281129989302264488322903470997529, s=51810181281739140198359475815786327534288399534719226071156574253455723117740, v=2709)

In [19]:
# send the transaction
send_store_txn = w3.eth.send_raw_transaction(signed_store_txn.rawTransaction)

# wait for the transaction to be mined
send_store_receipt = w3.eth.wait_for_transaction_receipt(send_store_txn)
send_store_receipt

AttributeDict({'transactionHash': HexBytes('0x79a778ba5f89a102853db07a34f88572b42c8c1942a96f956c11d2ba5ee57e1e'),
 'transactionIndex': 0,
 'blockNumber': 2,
 'blockHash': HexBytes('0xda0837f666553b82ef119e137e37b65b506bcc4b03c86cac818bacbad504ca80'),
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xe78A0F7E598Cc8b0Bb87894B0F60dD2a88d6a8Ab',
 'cumulativeGasUsed': 43724,
 'gasUsed': 43724,
 'contractAddress': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1

In [20]:
simple_storage.functions.retrieve().call()

15